## Keras Upgrade

In [ ]:
!pip install keras-nightly

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import random as rd
from sklearn.metrics import accuracy_score

import keras

In [ ]:
(train_x, train_y), (test_x, test_y) = keras.datasets.cifar10.load_data()

In [ ]:
labels = {0 : 'Airplane',
          1 : 'Automobile',
          2 : 'Bird',
          3 : 'Cat',
          4 : 'Deer',
          5 : 'Dog',
          6 : 'Frog',
          7 : 'Horse',
          8 : 'Ship',
          9 : 'Truck' }

In [ ]:
'''
Ctrl+Enter를 이용하여
반복 실행 해보자!
'''

id = rd.randrange(0,10000)

print(f'id = {id}')
print(f'다음 그림은 {labels[test_y[id][0]]} 입니다.')
plt.imshow(test_x[id])
plt.show()

In [ ]:
train_x.shape, train_y.shape

## **Transfer Learning**

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions

In [ ]:
keras.utils.clear_session()

base_model = ResNet50(include_top=False,
                      weights='imagenet',
                      input_shape=(32,32,3),
                      pooling='avg',
                      )

output_layer = keras.layers.Dense(10, activation='softmax')(base_model.output)

new_model = keras.models.Model(base_model.input, output_layer)
new_model.summary()

In [ ]:
new_model.layers

In [ ]:
for layer in new_model.layers[:-1] :
    layer.trainable = False

In [ ]:
new_model.compile(optimizer='adam', loss=keras.losses.sparse_categorical_crossentropy,
                   metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
es = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=4,
                   verbose=1,
                   restore_best_weights=True
                   )

In [ ]:
hist = new_model.fit(train_x, train_y, epochs=100, verbose=1,
                     validation_split=0.2, callbacks=[es]
                     )

* Evaluate

In [ ]:
new_model.evaluate(test_x, test_y)

* Inference

In [ ]:
y_pred = new_model.predict(test_x)

In [ ]:
y_pred.shape

In [ ]:
y_pred_arg = np.argmax(y_pred, axis=1)
test_y_arg = np.ravel(test_y)

### Visualize Inference

In [ ]:
'''
성능 확인을 위해
Ctrl+Enter를 이용하여
반복 실행 해보자!
'''

id = rd.randrange(0,10000)

print(f'id = {id}')
print(f'다음 그림은 {labels[test_y_arg[id]]} 입니다.')
print(f'모델의 예측 : {labels[y_pred_arg[id]]}')

prob = np.floor(y_pred[id]*100).tolist()
prob_dict = {}

for idx, prob in enumerate(prob) :
    prob_dict[ labels[idx] ] = prob

print('모델의 카테고리별 확률 : ')
print(prob_dict)

if test_y_arg[id] == y_pred_arg[id] :
    print('정답입니다')
else :
    print('틀렸어요')

plt.imshow(test_x[id].reshape([32,32,-1]))
plt.show()

In [ ]:
'''
틀린 것만 관찰해보자!

Ctrl+Enter를 이용하여
반복 실행 해보자!
'''

true_false = (test_y == y_pred)
f_id = np.where(true_false == False)[0]
f_n = len(f_id)

id = f_id[rd.randrange(0,f_n)]

print(f'id = {id}')
print(f'다음 그림은 {labels[test_y_arg[id]]} 입니다.')
print(f'모델의 예측 : {labels[y_pred_arg[id]]}')

prob = np.floor(y_pred[id]*100).tolist()
prob_dict = {}

for idx, prob in enumerate(prob) :
    prob_dict[ labels[idx] ] = prob

print('모델의 카테고리별 확률 : ')
print(prob_dict)

if test_y_arg[id] == y_pred_arg[id] :
    print('정답입니다')
else :
    print('틀렸어요')

plt.imshow(test_x[id].reshape([32,32,-1]))
plt.show()